In [1]:
import numpy as np
import time
import pandas as pd
import numpy.random as random
import matplotlib.pyplot as plt
import math
from gensim import corpora
from scipy import optimize
import scipy

In [2]:
#load data
data = pd.read_csv('poliblogs2008.csv')

### Ingest corpus to create documents and vocab

In [3]:
# load corpus
documents = corpora.MmCorpus('corpus.mm')
dictionary = corpora.Dictionary.load('dictionary')

In [4]:
vocab = dictionary.token2id

In [5]:
def init_stm(documents, settings): 
      
    K = settings['dim']['K']
    V = settings['dim']['V']
    A = settings['dim']['A']
    N = settings['dim']['N']
    
    #Random initialization
    mu = np.array([0]*(K-1))[:,None]
    sigma = np.zeros(((K-1),(K-1)))
    diag = np.diagonal(sigma, 0)
    diag.setflags(write=True)
    diag.fill(20)
    beta = random.gamma(.1,1, V*K).reshape(K,V)
    beta = (beta / beta.sum(axis=1)[:,None])
    lambd = np.zeros((N, (K-1)))
    
    #turn beta into a list and assign it for each aspect
    beta = [beta, beta] # FOR A=2
    kappa_initialized = init_kappa(documents, K, V, A, interactions=settings['kappa']['interactions'])
    
    #create model object
    model = {'mu':mu, 'sigma':sigma, 'beta': beta, 'lambda': lambd, 'kappa':kappa_initialized}
    
    return(model)

def init_kappa(documents, K, V, A, interactions): 
    # read in documents and vocab
    flat_documents = [item for sublist in documents for item in sublist]
    m = []

    total_sum = sum(n for _, n in flat_documents)

    for elem in flat_documents: 
        m.append(elem[1] / total_sum)

    m = np.log(m) - np.log(np.mean(m)) #logit of m


    #Defining parameters
    aspectmod = A > 1 # if there is more than one topical content variable
    if(aspectmod):
        interact = interactions # allow for the choice to interact
    else:
        interact = FALSE

    #Create the parameters object
    parLength = K + A * aspectmod + (K*A)*interact

    #create covariates. one element per item in parameter list.
    #generation by type because its conceptually simpler
    if not aspectmod & interact:
        covar = {'k': np.arange(K),
             'a': np.repeat(np.nan, parLength), #why parLength? 
             'type': np.repeat(1, K)}

    if(aspectmod & interact == False):
        covar = {'k': np.append(np.arange(K), np.repeat(np.nan, A)),
                 'a': np.append(np.repeat(np.nan, K), np.arange(A)), 
                 'type': np.append(np.repeat(1, K), np.repeat(2, A))}      
    if(interact):
        covar = {'k': np.append(np.arange(K), np.append(np.repeat(np.nan, A), np.repeat(np.arange(K), A))),
                 'a': np.append(np.repeat(np.nan, K), np.append(np.arange(A), np.repeat(np.arange(A), K))), 
                 'type': np.append(np.repeat(1, K), np.append(np.repeat(2, A),  np.repeat(3,K*A)))}

    kappa = {'out': {'m':m,
                     'params' : np.tile(np.repeat(0,V), (parLength, 1)),
                     'covar' : covar
                     #'kappasum':, why rolling sum?
                    }
            }

    return(kappa['out'])

In [6]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def softmax_weights(x, weight):
    """Compute softmax values for each sets of scores in x."""
    e_x = weight*np.exp(x - np.max(x))[:,None]
    return e_x / e_x.sum(axis=0)

def lhood(eta, mu, siginv, doc_ct, Ndoc, eta_long, beta_tuple, phi, theta, neta):
    
    #formula 
    #rewrite LSE to prevent overflow
    part1 = np.sum(doc_ct * (eta_long.max() + np.log(np.exp(eta_long - eta_long.max())@beta_tuple)))-np.sum(doc_ct)*scipy.special.logsumexp(eta)
    part2 = .5*(eta-mu)@siginv@(eta-mu)
    
    out = part2 - part1
    
    return -out

def grad(eta, mu, siginv, doc_ct,  Ndoc, eta_long, beta_tuple, phi, theta, neta):

    #formula
    part1 = np.delete(np.sum(phi * doc_ct,axis=1) - np.sum(doc_ct)*theta, neta)
    part2 = siginv@(eta-mu)

    return part2 - part1

In [7]:
def e_step(documents, mu, sigma, lambd, beta):
    #quickly define useful constants
    V = beta['beta'][0].shape[1] # ncol
    K = beta['beta'][0].shape[0] # nrow
    N = len(documents)
    A = len(beta['beta'])
    
    # 1) Initialize Sufficient Statistics 
    sigma_ss = np.zeros(((K-1),(K-1)))
    if A == 2: 
        beta_ss = [np.zeros((K,V)), np.zeros((K,V))]
    else:
        print('Error: Only two metadata columns allowed.')
    bound = np.repeat(0,N)
    #lambd = np.repeat(0,N)
    
    # 2) Precalculate common components
    sigobj = np.linalg.cholesky(sigma)
    sigmaentropy = np.sum(np.log(np.diag(sigobj)))
    siginv = np.linalg.inv(sigobj).T*np.linalg.inv(sigobj)
    
    # 3) Document Scheduling
    # For right now we are just doing everything in serial.
    # the challenge with multicore is efficient scheduling while
    # maintaining a small dimension for the sufficient statistics.
    
    mu = mu.flatten()


        #set parameters for one document (i)
    for i in range(N):

        eta=lambd[i]
        neta = len(eta)
        eta_long = np.insert(eta,-1,0)

        doc = documents[i]
        words = [x for x,y in doc]
        aspect = betaindex[i]
        beta_i = beta['beta'][aspect][:,[words]]


        #set document specs
        doc_ct = np.array([y for x,y in doc]) #count of words in document
        Ndoc = np.sum(doc_ct)

        # initial values
        beta_tuple = beta_i.reshape(K,beta_i.shape[2])
        theta = softmax(eta_long)
        phi = softmax_weights(eta_long, beta_tuple)

        optimize.fmin_bfgs(lhood,x0=eta,
                           args=(mu, siginv, Ndoc, doc_ct, eta_long, beta_tuple, phi, theta, neta),
                           fprime=grad)
        

        # update sufficient statistics

In [8]:
prevalence = 'blog'
content = 'rating'
num_topics = 10

# Setting control variables

In [9]:
def makeTopMatrix(x, data=None):
    return(data.loc[:,x])

xmat = makeTopMatrix(content, data)
yvar = xmat.astype('category')
yvarlevels = set(yvar)
betaindex = yvar.cat.codes
A = len(set(betaindex))

interactions = True #settings.kappa
verbose = True

init_type = "Random" #settings.init
ngroups = 1 #settings.ngroups
max_em_its = 15 #settings.convergence
emtol = 1e-5 #settings.convergence

#gamma_prior=("Pooled","L1") # settings.gamma.prior
#sigma_prior=0 #settings.sigma.prior
#kappa_prior=("L1","Jeffreys") # settings.kappa.prior

#Initialize parameters

settings = {
    'dim':{
        'K': num_topics, #number of topics
        'V' : len(dictionary), #number of words
        'A' : A, #dimension of topical content
        'N' : len(documents),
    },
    'kappa':{
        'interactions':True,
        'fixedintercept': True,
        'contrats': False,
        'mstep': {'tol':0.01, 'maxit':5}},
    'tau':{
        'mode': np.nan,
        'tol': 1e-5,
        'enet':1,
        'nlambda':250,
        'lambda.min.ratio':.001,
        'ic.k':2,
        'maxit':1e4},
    'init':{
        'mode':init_type, 
        'nits':20,
        'burnin':25,
        'alpha':50/num_topics,
        'eta':.01,
        's':.05,
        'p':3000},
    'convergence':{
        'max.em.its':max_em_its,
        'em.converge.thresh':emtol,
        'allow.neg.change':True,},
    'covariates':{
        'X':xmat,
        'betaindex':betaindex,
        'yvarlevels':yvarlevels,
        'formula': prevalence,},
    'gamma':{
        'mode':np.nan,
        'prior':np.nan,
        'enet':1, 
        'ic.k':2,
        'maxits':1000,},
    'sigma':{
        #'prior':sigma_prior,
        'ngroups':ngroups,},
}

In [10]:
def stm_control(documents, vocab, settings, model=None):
    
    ##########
    #Step 1: Initialize Parameters
    ##########
    
    #ngroups = settings$ngroups
    
    if model == None:
        print('Call init_stm()')
        model = init_stm(documents, settings) #initialize
    else: 
        model = model
        
    # unpack initialized model
    
    mu = model['mu']
    sigma = model['sigma']
    lambd = model['lambda'] 
    beta = {'beta': model['beta'],
            'kappa': model['kappa']}
    
    convergence = None
    
    #discard the old object
    del model
    
    betaindex = settings['covariates']['betaindex']
    
    #Pull out some book keeping elements
    betaindex = settings['covariates']['betaindex']
    
    ############
    #Step 2: Run EM
    ############
    
    t1 = time.process_time()

    #run the model
    e_step(documents, mu, sigma, lambd, beta)
        
    print("Completed E-Step ({} seconds). \n".format(math.floor((time.process_time()-t1))))


In [14]:
#e-step iteration for all N documents: 52 seconds

stm_control(documents, vocab, settings)

Call init_stm()
Optimization terminated successfully.
         Current function value: -422930997.405709
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -1985455425.834367
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -412138938.947743
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -618329848.916506
         Iterations: 7
         Function evaluations: 16
         Gradient evaluations: 16
Optimization terminated successfully.
         Current function value: -361983370.572363
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -5310328777.

Optimization terminated successfully.
         Current function value: -8518055044.800678
         Iterations: 6
         Function evaluations: 16
         Gradient evaluations: 16
Optimization terminated successfully.
         Current function value: -3659956957.577679
         Iterations: 3
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -83076185.015738
         Iterations: 3
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: -2543521223.698568
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -2512812443.020436
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -5848794590.068587
       

Optimization terminated successfully.
         Current function value: -459851848.056158
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -503611834.130259
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -6830574158.757608
         Iterations: 5
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -897332648.975451
         Iterations: 5
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -176546611.633201
         Iterations: 5
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -1710834242.309653
         

Optimization terminated successfully.
         Current function value: -155145335.849969
         Iterations: 5
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -243480791.083642
         Iterations: 3
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: -296558255.626745
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -3091555750.354324
         Iterations: 6
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -284118494.390213
         Iterations: 3
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: -2145860465.980257
         

Optimization terminated successfully.
         Current function value: -430086765.807408
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -692307962.167625
         Iterations: 4
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -426014053.694221
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -1612401635.275442
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -202351797.257184
         Iterations: 6
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -26628418663.509098
        

...

Optimization terminated successfully.
         Current function value: -305630814.695289
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -5254946366.621401
         Iterations: 4
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -4363274185.669473
         Iterations: 3
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -1187430481.311595
         Iterations: 4
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -270879321.672471
         Iterations: 3
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: -4054541415.082489
       

Optimization terminated successfully.
         Current function value: -445205778.511667
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -144088079.191654
         Iterations: 4
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: -829920569.199399
         Iterations: 5
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -103973380.988935
         Iterations: 3
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: -444724202.871144
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -843499995.832685
         It

Optimization terminated successfully.
         Current function value: -109753391.120755
         Iterations: 4
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: -86483855.334095
         Iterations: 3
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: -159203330.727865
         Iterations: 5
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -1076617465.876571
         Iterations: 4
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: -585772255.802479
         Iterations: 6
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -218713378.151320
         It

Optimization terminated successfully.
         Current function value: -110663420.237493
         Iterations: 4
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: -2980342720.899017
         Iterations: 6
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -192116107.689047
         Iterations: 5
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -749626785.314404
         Iterations: 5
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -160087652.879770
         Iterations: 5
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: -1989281288.632869
         

Optimization terminated successfully.
         Current function value: -543701291.545494
         Iterations: 6
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -85865048.485579
         Iterations: 3
         Function evaluations: 11
         Gradient evaluations: 11
Completed E-Step (49 seconds). 



From **A Model of Text for Experimentation in the Social Sciences (Roberts et al.)**:
- $\beta_k$ is a V-dimensional probability mass function that controls the frequency according to which terms are generated from that topic.

From **The Structural Topic Model and Applied Social Science (Roberts et al.)**:
- 'For the nonconjugate logistic normal variables in the E-step we use a Laplace approximation.'

## E-Step

### Likelihood Function 

$f(\hat{\eta}_{d}) \propto  - \frac{1}{2} (\eta_d-\mu_d)^T \sum^{-1}(\eta_d-\mu_d)+\big(\sum_v c_{d,v} log\sum_k \beta_{k,v} e^{\eta_{d,k}}- W_d log \sum_k e^{\eta_{d,k}}\big)$

### Gradient of the Likelihood

$\nabla f(\eta_d)_k = (\sum c_{d,v} \langle \phi_{d,v,k} \rangle) - W_d\theta_{d,k}-\big(\sum^{-1}(\eta_d-\mu_d)\big)_k$

with $\theta_{d,k} = \frac{exp(\eta)}{\sum exp(\eta)}$


and $\langle\phi_{d,k,v}\rangle = \frac{exp(\eta_{d,k}) \beta_{d,v,k}}{\sum_k exp(\eta_{d,k}) \beta_{d,v,k}} $

### Variational posterior: 
$q(\eta_d) \sim N(\hat{\eta}_d, -\nabla^2 f(\hat{\eta}_d)^{-1})$

# Solve for Hessian/Phi/Bound returning the result

In [249]:
def hpb(estep_out, eta, mu, siginv, doc_ct, Ndoc, eta_long, beta_tuple, phi, theta, neta):
    # 1. divide beta by expeta (column-wise)
    #    beta with dimension (VxK), expeta with dimension K
    # copy to mess with 
    beta_temp = beta_tuple
    #column-wise multiplication of beta and expeta
    beta_temp = beta_temp*eta_long
    beta_temp = beta_temp*(np.sqrt(doc_ct) / np.sum(beta_temp, axis=0))
    
    
    
    

In [273]:
#define constant vals 
a = random.gamma(.1,1, 100*10).reshape(100,10)
a = (a / a.sum(axis=1)[:,None])
b = np.zeros((1, 10))
c = np.arange(1,101,1)
t = softmax(b)
sigma = np.diag(np.repeat(20,(10-1)))
# 2) Precalculate common components
sigobj = np.linalg.cholesky(sigma)
sigmaentropy = np.sum(np.log(np.diag(sigobj)))
siginv = np.linalg.inv(sigobj).T*np.linalg.inv(sigobj)

#operation
ab = (a * b)+1 # +1 to avoid zero-divide
ab_temp = ab*(np.sqrt(c)[:,None]/np.sum(ab,axis=0)) # with shape (VxK)
h = (ab_temp.T@ab_temp)-(sum(c)*(t*t.T)) # hessian with shape KxK
ab = ab.T * np.sqrt(c) #should be equal to phi ??? with shape KxV
np.fill_diagonal(h, np.diag(h)-np.sum(ab, axis=1)-np.sum(c)*t) #altered diagonal of h
# drop last row and columns
h = np.delete(h,9,0)
h = np.delete(h,9,1)
h = h + siginv # at this point, the hessian is complete

# Invert h via cholesky decomposition

array([[-10770.9079471,  -5049.495    ,  -5049.495    ,  -5049.495    ,
         -5049.495    ,  -5049.495    ,  -5049.495    ,  -5049.495    ,
         -5049.495    ],
       [ -5049.495    , -10770.9079471,  -5049.495    ,  -5049.495    ,
         -5049.495    ,  -5049.495    ,  -5049.495    ,  -5049.495    ,
         -5049.495    ],
       [ -5049.495    ,  -5049.495    , -10770.9079471,  -5049.495    ,
         -5049.495    ,  -5049.495    ,  -5049.495    ,  -5049.495    ,
         -5049.495    ],
       [ -5049.495    ,  -5049.495    ,  -5049.495    , -10770.9079471,
         -5049.495    ,  -5049.495    ,  -5049.495    ,  -5049.495    ,
         -5049.495    ],
       [ -5049.495    ,  -5049.495    ,  -5049.495    ,  -5049.495    ,
        -10770.9079471,  -5049.495    ,  -5049.495    ,  -5049.495    ,
         -5049.495    ],
       [ -5049.495    ,  -5049.495    ,  -5049.495    ,  -5049.495    ,
         -5049.495    , -10770.9079471,  -5049.495    ,  -5049.495    ,
         -5

## M-Step


### Topical Content Model Update

### Topical Prevalence Model Update

### Global Covariance Update